# Procesamiento del lenguaje natural - 2025 - B4 - Desafio 2
**Inteligencia Artificial - CEIA - FIUBA**

## Autor

- **Mendoza Dante**.
- **SIU: e2206**.

**Nota:** tome como base el codigo compartido por los docentes. Elegi un dataset encontrado en el sitio de kaggle: https://www.kaggle.com/datasets/paultimothymooney/poetry/data

### Actividades.
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [ ]:
!pip install gensim

In [ ]:
# ========================================================================================
# Word Embeddings
# ========================================================================================

import os
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from sklearn.manifold import TSNE
import plotly.express as px
import kagglehub

# ========================================================================================
# Descargo dataset de poesía
# ========================================================================================
path = kagglehub.dataset_download("paultimothymooney/poetry")
print("Dataset descargado en:", path)

# ========================================================================================
# Cargo los poemas .txt
# ========================================================================================
poems = []
authors = []

for root, dirs, filenames in os.walk(path):
    for fname in filenames:
        if fname.endswith(".txt"):
            full_path = os.path.join(root, fname)
            try:
                with open(full_path, "r", encoding="utf-8", errors="ignore") as f:
                    text = f.read().strip()
                    if text:
                        poems.append(text)
                        authors.append(os.path.basename(root))
            except Exception as e:
                print(f"Error leyendo {fname}: {e}")

df = pd.DataFrame({"author": authors, "poem": poems})
print("Cantidad de poemas cargados:", len(df))
print(df.head(4))

# ========================================================================================
# Armo los tokens
# ========================================================================================
sentence_tokens = []
for _, row in df.iterrows():
    sentence_tokens.append(text_to_word_sequence(row["poem"]))

print("\nEjemplo de tokens (primeras 10 palabras de los 2 primeros versos):")
for i, tokens in enumerate(sentence_tokens[:2]):
    print(f"Verso {i+1}: {tokens[:10]} ...")

# ========================================================================================
# Entrenamiento del modelo Word2Vec
# ========================================================================================
w2v_model = Word2Vec(
    sentences=sentence_tokens,
    vector_size=100,
    window=5,
    min_count=3,
    workers=2,
    sg=1,
    epochs=30
)

print("Tamaño del vocabulario:", len(w2v_model.wv.index_to_key))

# ========================================================================================
# Similitudes con palabras de interés
# ========================================================================================
PALABRAS_DE_INTERES = ["love", "death", "life", "dream", "heart"]

for p in PALABRAS_DE_INTERES:
    if p in w2v_model.wv:
        print(f"\nPalabras más similares a '{p}':")
        print(w2v_model.wv.most_similar(p, topn=5))
    else:
        print(f"\nLa palabra '{p}' no está en el vocabulario")

# ========================================================================================
# Visualización de embeddings en 2D
# ========================================================================================
def reducir_dimensiones(model, num_dimensions=2):
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)
    tsne = TSNE(n_components=num_dimensions, random_state=42, perplexity=30)
    vectors = tsne.fit_transform(vectors)
    return vectors, labels

vecs, labels = reducir_dimensiones(w2v_model)

MAX_WORDS = 200
fig = px.scatter(
    x=vecs[:MAX_WORDS, 0], y=vecs[:MAX_WORDS, 1],
    text=labels[:MAX_WORDS],
    title="Embeddings de palabras en Poesía"
)
fig.show(renderer="colab")


100%|██████████| 2.00M/2.00M [00:00<00:00, 140MB/s]

Extracting files...
Dataset descargado en: /root/.cache/kagglehub/datasets/paultimothymooney/poetry/versions/16
Cantidad de poemas cargados: 49
  author                                               poem
0     16  Now greetings to the world! Standing at this l...
1     16  Ghost in the mirror\nI knew your face once, bu...
2     16  [:Nicki Minaj]\nYoung money [Verse 1: Jason De...
3     16  I saw you walking by his side heard you whispe...



Ejemplo de tokens (primeras 10 palabras de los 2 primeros versos):
Verso 1: ['now', 'greetings', 'to', 'the', 'world', 'standing', 'at', 'this', 'liquor', 'store'] ...
Verso 2: ['ghost', 'in', 'the', 'mirror', 'i', 'knew', 'your', 'face', 'once', 'but'] ...
Tamaño del vocabulario: 14334

Palabras más similares a 'love':
[('equal', 0.6727054715156555), ("sho'", 0.6325149536132812), ("'nuff", 0.5976601839065552), ('owww', 0.5854165554046631), ('cowboy', 0.5803887844085693)]

Palabras más similares a 'death':
[('dialogue', 0.6265811324119568), ('apparently', 0.6122438311576843), ('vengeance', 0.6000005602836609), ('pledged', 0.5900827050209045), ('certificate', 0.576103687286377)]

Palabras más similares a 'life':
[('goals', 0.6263910531997681), ('taxicabs', 0.6224331855773926), ('bossed', 0.5928923487663269), ('damned', 0.5763543248176575), ('timbuck', 0.5750008225440979)]

Palabras más similares a 'dream':
[('deathless', 0.624456524848938), ("look's", 0.6108185052871704), ('awh', 0.558

Conclusiones

El entrenamiento con el dataset de poetry permitió observar cómo el modelo Word2Vec (Skip-gram) captura relaciones semánticas entre palabras en el contexto de poemas.

Para la palabra "love", aparecieron términos con bastante ruido ("sho'", "ove"), lo que refleja la presencia de palabras mal tokenizadas. Quizas una normalización ayude a mejorar este aspecto.

La palabra "death" se asoció con conceptos como "certificate", "vengeance" o "marked", lo que refleja un contexto semántico coherente relacionado con violencia, consecuencias o finalidades.

En el caso de "life", se relacionó con "live", "overprotected" y "threatening", mostrando que la vida en este corpus se vincula con protección y peligro, lo cual se lo ve relacionado.

La palabra "dream" apareció próxima a "deathless", "lasting" y "flung", lo que sugiere una visión trascendental y metafórica del soñar.

Por utlimo, "heart" mostró asociaciones con "pounding", "bitterness" y "photographs", reflejando metáforas emocionales y sentimentales.

La visualización en 2D permitió agrupar palabras según su contexto de uso. Aunque se observa ruido en los embeddings debido a la falta de limpieza del dataset, el grafico es mejorable si se realiza una normalización y limitar la cantidad de palabras. Finalmente, veo que el modelo logra capturar relaciones semánticas relevantes y coherentes en el dominio poético, aunque con limitaciones derivadas de la calidad del dataset.